In [ ]:
!pip install ortools

In [198]:
import random
from ortools.sat.python import cp_model

def generate_valid_path(size):
    path = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    current = (random.randint(0, size-1), random.randint(0, size-1))
    path.append(current)
    used = set(path)

    while len(path) < size * size:
        possible_moves = [(current[0] + dx, current[1] + dy) for dx, dy in directions
                          if 0 <= current[0] + dx < size and 0 <= current[1] + dy < size
                          and (current[0] + dx, current[1] + dy) not in used]
        if not possible_moves:
            path, used = [], set()
            current = (random.randint(0, size-1), random.randint(0, size-1))
            path.append(current)
            used.add(current)
        else:
            current = random.choice(possible_moves)
            path.append(current)
            used.add(current)
    return path

def solve_rikudo_puzzle(size, path):
    model = cp_model.CpModel()

    # Create variables for each grid position
    grid = {}
    for i, pos in enumerate(path):
        grid[pos] = model.NewIntVar(1, size * size, f'cell_{pos}')

    # Ensure the sequence is correct
    for i in range(1, len(path)):
        model.Add(grid[path[i]] == grid[path[i-1]] + 1)

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        solution_grid = [[0] * size for _ in range(size)]
        for pos, var in grid.items():
            solution_grid[pos[0]][pos[1]] = solver.Value(var)
        return solution_grid
    else:
        return None

# Example usage
size = 6
random_path = generate_valid_path(size)
solution = solve_rikudo_puzzle(size, random_path)
if solution:
    for row in solution:
        print(' '.join(f"{val:2d}" for val in row))
else:
    print("No solution found.")


12 11 10  8  7  6
13 14  9 27  4  5
15 16 28 26  3  2
17 24 25 29  1 31
23 18 20 35 30 32
22 21 19 36 34 33
